# Mapping Data with Folium

![](images/folium.png)

Below, we will go through a brief introduction to the **Folium** library.  This is a nice way to build interactive visuzlizations.  We will be executing these in the jupyter notebooks, however they are easily output as `.html` files ready to be served.  To begin, let's make sure we have folium installed.  

In [14]:
import folium
import pandas as pd
#pip install folium

We can make basic maps centered at any geolocation.  For example, below we create a basic map of Portland, Oregon.  

In [2]:
m = folium.Map(location=[45.5236, -122.6750])

In [3]:
m

In [4]:
m.save('index_map.html')

We can add arguments that include changing the style of the map and the initial zoom level.

In [15]:
folium.Map(
    location=[45.5236, -122.6750],
    tiles='Stamen Toner',
    zoom_start=13
)

We can use the `popup` argument to include information to be displayed at specified marker locations.

In [16]:

m = folium.Map(
    location=[45.372, -121.6972],
    zoom_start=12,
    tiles='Stamen Terrain'
)



folium.Marker([45.3288, -121.6625], popup='<i>Mt. Hood Meadows</i>').add_to(m)
folium.Marker([45.3311, -121.7113], popup='<b>Timberline Lodge</b>').add_to(m)
m

We can even include `markdown` syntax and icons.

In [17]:
m = folium.Map(
    location=[45.372, -121.6972],
    zoom_start=12,
    tiles='Stamen Terrain'
)

folium.Marker(
    location=[45.3288, -121.6625],
    popup='Mt. Hood Meadows',
    icon=folium.Icon(icon='cloud')
).add_to(m)

folium.Marker(
    location=[45.3311, -121.7113],
    popup='Timberline Lodge',
    icon=folium.Icon(color='green')
).add_to(m)

folium.Marker(
    location=[45.3300, -121.6823],
    popup='Some Other Location',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

In [18]:
m

We can manually control radii for markers of interest.  Below, we plot two circles at specific locations.

In [9]:
m = folium.Map(
    location=[45.5236, -122.6750],
    tiles='Stamen Toner',
    zoom_start=13
)

folium.Circle(
    radius=100,
    location=[45.5244, -122.6699],
    popup='The Waterfront',
    color='crimson',
    fill=False,
).add_to(m)

folium.CircleMarker(
    location=[45.5215, -122.6261],
    radius=50,
    popup='Laurelhurst Park',
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(m)


In [10]:
m

### Mapping Bike Data

Now, we will use a dataset from NYC's citibike data.  Our goal is to compare incoming and outgoing traffic at given stations depending on the time of day.  

In [19]:
folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="CartoDB dark_matter")
marker = folium.CircleMarker(location=[40.738, -73.98])
marker.add_to(folium_map)

In [20]:
folium_map

In [13]:
bikes = pd.read_csv('../data/201306-citibike-tripdata.zip', compression = 'zip')

ValueError: ('Multiple files found in compressed zip file %s', "['201306-citibike-tripdata.csv', '__MACOSX/', '__MACOSX/._201306-citibike-tripdata.csv']")

In [ ]:
bikes.info()

In [ ]:
bikes['starttime'] = pd.to_datetime(bikes['starttime'])
bikes['stoptime'] = pd.to_datetime(bikes['stoptime'])
bikes['hour'] = bikes['starttime'].map(lambda x: x.hour)
bikes['ehour'] = bikes['stoptime'].map(lambda x: x.hour)

In [ ]:
bikes.head()

In [ ]:
locations = bikes.groupby('start station id').first()

In [ ]:
locations = locations.loc[:, ["start station latitude", "start station longitude", "start station name"]]

In [ ]:
subset = bikes[bikes["hour"]==10]

In [ ]:
dept_counts = subset.groupby("start station id").count()

In [ ]:
dept_counts = dept_counts.iloc[:, [0]]

In [ ]:
dept_counts.columns = ["Departure Counts"]

### Problem

Repeat the above for arrivals, in anticipation of joining the two for our map.

In [ ]:
bikes['hour'] = bikes['starttime'].map(lambda x: x.hour)
locations = locations.loc[:, ["start station latitude", "start station longitude", "start station name"]]
subset = bikes[bikes["hour"]==18]
dept_counts = subset.groupby("start station id").count()
dept_counts = dept_counts.iloc[:, [0]]
dept_counts.columns = ["Departure Counts"]


locations2 = bikes.groupby('end station id').first()
locations2 = locations2.loc[:, ["end station latitude", "end station longitude", "end station name"]]
subset = bikes[bikes["ehour"]==18]
arr_counts = subset.groupby("end station id").count()
arr_counts = arr_counts.iloc[:, [0]]
arr_counts.columns = ["Arrival Counts"]

In [ ]:
trip_counts = dept_counts.join(locations).join(arr_counts)

In [ ]:
trip_counts.head()

In [ ]:
for index, row in trip_counts.iterrows():
    
    net_departures = (row["Departure Counts"]-row["Arrival Counts"])
    
    radius = net_departures/7
    
    if net_departures>0:
        color="#E37222" # tangerine
    else:
        color="#0A8A9F" # teal
    
    folium.CircleMarker(location=(row["start station latitude"],
                                  row["start station longitude"]),
                        radius=radius,
                        color=color,
                        fill=True).add_to(folium_map)

In [ ]:
folium_map

In [ ]:
popup_text = """{}<br>
                total departures: {}<br> 
                total arrivals: {}<br>
                net departures: {}"""


popup_text = popup_text.format(row["start station name"],
                               row["Arrival Counts"],
                               row["Departure Counts"],
                               net_departures)

In [ ]:
for index, row in trip_counts.iterrows():
    net_departures = (row["Departure Counts"]-row["Arrival Counts"])
    radius = net_departures/7
    if net_departures>0:
        color="#E37222" # tangerine
    else:
        color="#0A8A9F" # teal
    
    folium.CircleMarker(location=(row["start station latitude"],
                                  row["start station longitude"]),
                        radius=radius,
                        color=color,
                        fill=True, popup = popup_text).add_to(folium_map)

In [ ]:
folium_map

### PROBLEM

Compare this image to that of when people are leaving work.  Doe you see what you expect?  What does this tell you about movement in the city?

In [1]:
import pandas as pd
import folium
health = pd.read_csv('500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2017_release_.csv')

In [2]:
health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27204 entries, 0 to 27203
Data columns (total 63 columns):
StateAbbr                 27204 non-null object
PlaceName                 27204 non-null object
PlaceFIPS                 27204 non-null int64
TractFIPS                 27204 non-null int64
Place_TractID             27204 non-null object
Population2010            27204 non-null int64
ACCESS2_CrudePrev         27203 non-null float64
ACCESS2_Crude95CI         27203 non-null object
ARTHRITIS_CrudePrev       27204 non-null float64
ARTHRITIS_Crude95CI       27204 non-null object
BINGE_CrudePrev           27204 non-null float64
BINGE_Crude95CI           27204 non-null object
BPHIGH_CrudePrev          27204 non-null float64
BPHIGH_Crude95CI          27204 non-null object
BPMED_CrudePrev           27204 non-null float64
BPMED_Crude95CI           27204 non-null object
CANCER_CrudePrev          27204 non-null float64
CANCER_Crude95CI          27204 non-null object
CASTHMA_CrudePrev       

In [81]:
health.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Population2010,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,PHLTH_Crude95CI,SLEEP_CrudePrev,SLEEP_Crude95CI,STROKE_CrudePrev,STROKE_Crude95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,Geolocation,lat,long
0,AL,Birmingham,107000,1073000100,0107000-01073000100,3042,23.9,"(21.2, 27.2)",32.5,"(31.5, 33.6)",...,"(18.2, 21.2)",50.4,"(49.1, 51.6)",5.5,"( 5.0, 6.0)",30.2,"(24.1, 36.3)","(33.57943283260, -86.7228323926)",33.579432,-86.72283
1,AL,Birmingham,107000,1073000300,0107000-01073000300,2735,28.8,"(25.4, 32.4)",31.3,"(30.0, 32.4)",...,"(18.5, 22.0)",49.4,"(47.8, 50.8)",5.7,"( 5.1, 6.4)",33.3,"(25.6, 41.0)","(33.54282086860, -86.7524339780)",33.542820,-86.75243
2,AL,Birmingham,107000,1073000400,0107000-01073000400,3338,26.1,"(22.6, 29.9)",34.6,"(33.2, 35.9)",...,"(19.1, 23.1)",52.5,"(51.2, 53.7)",6.5,"( 5.8, 7.2)",36.0,"(28.9, 43.5)","(33.56324496330, -86.7640474064)",33.563244,-86.76404
3,AL,Birmingham,107000,1073000500,0107000-01073000500,2864,28.1,"(24.6, 32.0)",37.8,"(36.3, 39.2)",...,"(21.0, 25.4)",52.6,"(51.1, 54.0)",8.2,"( 7.3, 9.2)",40.2,"(31.1, 48.9)","(33.54424045940, -86.7749130719)",33.544240,-86.77491
4,AL,Birmingham,107000,1073000700,0107000-01073000700,2577,31.8,"(27.0, 36.7)",38.5,"(37.1, 39.9)",...,"(23.5, 28.2)",54.1,"(52.3, 55.6)",9.6,"( 8.5, 10.7)",45.8,"(35.2, 56.1)","(33.55254061390, -86.8016893706)",33.552540,-86.80168


In [3]:
health.Geolocation[0]

'(33.57943283260, -86.7228323926)'

In [27]:
health.Geolocation[0][1:6]

'33.57'

In [30]:
health.Geolocation[0][17:23]

'-86.72'

In [17]:
lat = []
long = []
for pair in health.Geolocation:
    lat.append(float(pair[1:10]))
    long.append(float(pair[17:26]))

In [18]:
health['lat'] = lat
health['long'] = long

In [19]:
m = folium.Map([health.lat[0], health.long[0]])

In [20]:
m

In [22]:
health.head(1)

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Population2010,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,PHLTH_Crude95CI,SLEEP_CrudePrev,SLEEP_Crude95CI,STROKE_CrudePrev,STROKE_Crude95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,Geolocation,lat,long
0,AL,Birmingham,107000,1073000100,0107000-01073000100,3042,23.9,"(21.2, 27.2)",32.5,"(31.5, 33.6)",...,"(18.2, 21.2)",50.4,"(49.1, 51.6)",5.5,"( 5.0, 6.0)",30.2,"(24.1, 36.3)","(33.57943283260, -86.7228323926)",33.579432,-86.72283


In [30]:
new_york = health[health['StateAbbr'] == 'NY']

In [70]:
m = folium.Map([new_york.lat.iloc[0], new_york.long.iloc[0]], zoom_start=7)

In [71]:
m

In [74]:
albany = new_york[new_york['PlaceName'] == 'Albany']

In [104]:
alb = folium.Map([albany.lat.iloc[0], albany.long.iloc[0]], tiles="Mapbox")

ValueError: You must pass an API key if using Cloudmade or non-default Mapbox tiles.

In [101]:
alb

In [102]:
for i in range(len(albany.lat)):
    folium.CircleMarker([albany['lat'].iloc[i], albany['long'].iloc[i]],
                        radius = albany.OBESITY_CrudePrev.iloc[i],
                   color = 'red', fill = True, popup = 'Obesity rate: {:.2f}'.format(albany.OBESITY_CrudePrev.iloc[i])).add_to(alb)

In [103]:
alb